# Sales Forecast using Machine Learning

This notebook outlines the process of building a machine learning model to forecast weekly sales for a retail company. The solution achieved **3rd place** in the "Predykcja sprzedaży z pomocą Machine Learning" Kaggle competition.


## 1. Project Overview

The primary goal of this project is to accurately predict `weekly_sales` for various store-department combinations. The solution leverages historical sales data along with supplementary information about stores, economic indicators, and promotional markdowns. The modeling approach involves extensive feature engineering, the use of gradient boosting models (XGBoost and CatBoost), and hyperparameter optimization to achieve high predictive accuracy.

## 2. Data Loading and Initial Setup

First, we'll load the necessary libraries and the datasets. The data is split into multiple files: historical sales, features (like temperature and fuel price), and store information.

### 2.1. Libraries

We begin by importing the essential Python libraries for data manipulation, visualization, and modeling.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import catboost as ctb
from functools import partial
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import eli5
from eli5.sklearn import PermutationImportance
import scikitplot as skplt

# Set default plot size
plt.rcParams["figure.figsize"] = (15, 5)

### 2.2. Data Loading

Here we load the training, testing, features, and store datasets. We also perform initial data type conversions for date columns.

In [ ]:
# Load datasets from the same directory structure as the original notebook
df_sales_train = pd.read_hdf("../input/sales_train.h5")
df_sales_test = pd.read_hdf("../input/sales_test.h5")
df_features_train = pd.read_hdf("../input/features_train.h5")
df_features_test = pd.read_hdf("../input/features_test.h5")
df_store = pd.read_csv("../input/stores_data.csv")

# Convert date columns to datetime objects
df_sales_train["date"] = pd.to_datetime(df_sales_train["date"], format='%d/%m/%Y')
df_sales_test["date"] = pd.to_datetime(df_sales_test["date"], format='%d/%m/%Y')
df_features_train["date"] = pd.to_datetime(df_features_train["date"], format='%d/%m/%Y')
df_features_test["date"] = pd.to_datetime(df_features_test["date"], format='%d/%m/%Y')

# Prepare store data
df_store.columns = ["store", "type", "size"]
df_store['type_cat'] = df_store['type'].factorize()[0]
df_store.drop(columns=['type'], inplace=True)

## 3. Feature Engineering

This is a critical step where we create new features to improve model performance. A key part of this process is the `simple_feature_engineering` function, which encapsulates all our feature creation logic.

### 3.1. Target Variable Transformation

To stabilize variance and handle the positive nature of sales data, we apply a logarithmic transformation to the `weekly_sales` target variable. An offset is used to handle zero or negative sales values if they exist.

In [ ]:
def log_transform_target(y, offset):
    """Applies a log transformation to the target variable."""
    return np.log(y - offset)

def inverse_log_transform_target(log_y, offset):
    """Reverses the log transformation."""
    return np.exp(log_y) + offset

# The offset is based on the minimum value in the training set to avoid log(0) or log(negative)
offset = df_sales_train['weekly_sales'].min() - 1
df_sales_train['log_weekly_sales'] = df_sales_train['weekly_sales'].map(lambda x: log_transform_target(x, offset))

### 3.2. Comprehensive Feature Creation

The `simple_feature_engineering` function creates a rich set of features, including:
- Time-based features (month, year, week, etc.).
- A unique identifier for each store-department combination.
- Aggregated sales statistics from the previous year.
- Store and department-level sales statistics (mean, std, median).

In [ ]:
def simple_feature_engineering(df, is_train=True, train_df=None):
    """
    Creates a set of new features from the existing data.
    `is_train` controls whether to compute stats (should only be done on training data).
    `train_df` is the original training dataframe, used to compute stats for the test set.
    """
    if train_df is None:
        train_df = df

    # Date features
    df['month'] = df["date"].dt.month
    df['year'] = df["date"].dt.year
    df['week'] = df["date"].dt.isocalendar().week
    df["dayofweek"] = df["date"].dt.dayofweek
    df["dayofyear"] = df["date"].dt.dayofyear

    # Interaction feature
    df['id-store-dept'] = df['store'].astype(str) + '_' + df['dept'].astype(str)

    # Store-level features from the training set
    temp_sum_dept = train_df.groupby('store')['dept'].nunique()
    df['sum_dept'] = df['store'].map(temp_sum_dept)

    # Merge store information (size, type)
    df = pd.merge(df, df_store, on=['store'], how='left')

    # --- Lagged and Statistical Features (computed from training data) ---
    df["year_minus_1"] = df["year"] - 1

    # Previous year's weekly average sales
    df_sales_group = train_df.groupby(["id-store-dept", "year", "week"]).agg(PY_week_log_sales=('log_weekly_sales', 'mean')).reset_index()
    df_sales_group = df_sales_group.rename(columns={'year': 'year_minus_1'})
    df = pd.merge(df, df_sales_group, on=['id-store-dept', 'year_minus_1', 'week'], how='left')

    # Previous year's monthly average sales
    df_sales_group_2 = train_df.groupby(["id-store-dept", "year", "month"]).agg(PY_month_log_sales=('log_weekly_sales', 'mean')).reset_index()
    df_sales_group_2 = df_sales_group_2.rename(columns={'year': 'year_minus_1'})
    df = pd.merge(df, df_sales_group_2, on=['id-store-dept', 'year_minus_1', 'month'], how='left')

    # Overall store sales statistics
    df_store_stats = train_df.groupby("store")['log_weekly_sales'].agg(['mean', 'std', 'median', 'size']).reset_index()
    df_store_stats.columns = ['store', 'store_mean', 'store_std', 'store_median', 'store_size']
    df = pd.merge(df, df_store_stats, on=['store'], how='left')

    # Overall department-in-store sales statistics
    df_store_stats_dept = train_df.groupby(['store', 'dept'])['log_weekly_sales'].agg(['mean', 'std', 'median', 'size']).reset_index()
    df_store_stats_dept.columns = ['store', 'dept', 'dept_mean', 'dept_std', 'dept_median', 'dept_size']
    df = pd.merge(df, df_store_stats_dept, on=['store', 'dept'], how='left')

    # Rolling mean feature (calculated on the original 'weekly_sales')
    # This is calculated separately as it's a forward-looking calculation within a group
    if is_train:
        df["rolling_mean_3"] = df.groupby(["store", "dept"])["weekly_sales"].transform(lambda x: x.rolling(3, min_periods=1).mean())
    else:
        # For test set, we can't compute a rolling mean. We will fill it.
        df["rolling_mean_3"] = -1

    # Fill any remaining missing values
    df.fillna(-1, inplace=True)

    return df

# Apply feature engineering to the training set
df_sales_train = simple_feature_engineering(df_sales_train, is_train=True)

# Apply feature engineering to the test set, using stats from the training set
df_sales_test = simple_feature_engineering(df_sales_test, is_train=False, train_df=df_sales_train)

# --- Merge external features (Temperature, CPI, etc.) ---
feat_vars_train = ['store', 'date', 'temperature', 'fuel_price', 'cpi', 'unemployment']
df_sales_train = pd.merge(df_sales_train, df_features_train[feat_vars_train], on=['store', 'date'], how='left')

# For the test set, we merge the external features based on the date
# If features for a specific test date aren't available, we'll forward-fill them later
df_sales_test = pd.merge(df_sales_test, df_features_test[feat_vars_train], on=['store', 'date'], how='left')

# Fill any NaNs created by merges (e.g., if test dates don't have features)
df_sales_train.fillna(-1, inplace=True)
df_sales_test.fillna(-1, inplace=True)

## 4. Model Training and Evaluation

With our feature-rich dataset, we now move to the modeling phase. We'll use two powerful gradient boosting models: **XGBoost** and **CatBoost**. We will also use GPU acceleration for speed.

### 4.1. Model and Evaluation Functions

We define a set of helper functions to streamline model training, cross-validation, and prediction.

In [ ]:
def get_models(xgb_params, ctb_params):
    """Returns a list of models to be trained."""
    return [
        ('xgb', xgb.XGBRegressor(**xgb_params)),
        ('ctb', ctb.CatBoostRegressor(**ctb_params))
    ]

def fit_and_predict(model, X_train, y_train, X_test, offset):
    """Fits the model on log-transformed data and makes predictions."""
    y_train_log = log_transform_target(y_train, offset)
    model.fit(X_train, y_train_log)
    y_pred_log = model.predict(X_test)
    return inverse_log_transform_target(y_pred_log, offset)

### 4.2. Feature Importance Analysis

Before full training, we analyze feature importance using `eli5`'s Permutation Importance on a sample of the data. This helps us understand which features are most influential and can guide feature selection. We use a base model for this analysis.

In [ ]:
# Define a temporary sample for faster importance calculation
temp_df_train = df_sales_train.sample(n=50000, random_state=42) 

# Define all potential features, excluding identifiers and target variables
all_feats = [col for col in temp_df_train.columns if col not in 
             ['id', 'weekly_sales', 'log_weekly_sales', 'date', 'year_minus_1', 'id-store-dept']]

def plot_feature_importance(X, y, offset, features, model):
    """Calculates and plots feature importance."""
    y_log = log_transform_target(y, offset)
    
    # Use Permutation Importance for a more robust measure
    perm = PermutationImportance(model, random_state=0, n_iter=2).fit(X, y_log)
    
    # Display results
    display(eli5.show_weights(perm, feature_names=features, top=30))

# Define base models for importance check
base_xgb = xgb.XGBRegressor(n_estimators=100, random_state=0, tree_method='gpu_hist')
base_ctb = ctb.CatBoostRegressor(n_estimators=100, random_state=0, verbose=False, task_type='GPU')

# Get feature values
X_temp = temp_df_train[all_feats].values
y_temp = temp_df_train['weekly_sales'].values

# XGBoost Feature Importance
print("--- XGBoost Feature Importance ---")
plot_feature_importance(X_temp, y_temp, offset, all_feats, base_xgb)

In [ ]:
# CatBoost Feature Importance
print("\n--- CatBoost Feature Importance ---")
plot_feature_importance(X_temp, y_temp, offset, all_feats, base_ctb)

# Based on the feature importance analysis, we select a curated list of the most impactful features.
# This reduces model complexity and training time.
final_feats = [
    'dept_mean',
    'rolling_mean_3',
    'PY_week_log_sales',
    'dept_median',
    'PY_month_log_sales',
    'dept_std',
    'week',
    'dept',
    'dayofyear',
    'dept_size',
    'fuel_price',
    'year',
    'temperature',
    'month',
    'size',
    'cpi',
    'store_size',
    'store_median'
]

print(f"\nSelected {len(final_feats)} features for the final model.")

## 5. Hyperparameter Tuning with Hyperopt

To find the optimal hyperparameters for our models, we use **Hyperopt**, a library for Bayesian optimization. The code for this is shown below but commented out, as it is computationally expensive. We will use the optimized parameters found during the competition.

In [ ]:
# The following parameters are the result of the hyperparameter tuning from the competition
xgb_best_params = {
    'colsample_bytree': 0.8057199686605526,
    'learning_rate': 0.16139979310707386,
    'max_depth': 14,
    'random_state': 4150,
    'subsample': 0.939396580073127,
    'n_estimators': 100, # Kept low for speed, can be increased with early stopping
    'tree_method': 'gpu_hist',
    'gpu_id': 0
}

ctb_best_params = {
    'depth': 14,
    'random_state': 10000,
    'n_estimators': 100, # Kept low for speed
    'verbose': False,
    'task_type': 'GPU',
    'devices': '0'
}

print("Optimized hyperparameters have been loaded.")

# --- Example Hyperopt implementation (run during competition, not executed here) ---
# space_xgb = {
#     'max_depth': hp.quniform('max_depth', 5, 20, 1),
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.8, 1.0),
#     # ... other params
# }

# def objective_xgb(space):
#     X, y = df_sales_train[final_feats].values, df_sales_train['weekly_sales'].values
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#     model_params = {
#         'max_depth': int(space['max_depth']),
#         'colsample_bytree': space['colsample_bytree'],
#         # ... other params
#         'n_estimators': 100, 'tree_method': 'gpu_hist'
#     }
#     y_pred = fit_and_predict(xgb.XGBRegressor(**model_params), X_train, y_train, X_test, offset)
#     score = mean_absolute_error(y_test, y_pred)
#     return {'loss': score, 'status': STATUS_OK}

# trials = Trials()
# best_params = fmin(fn=objective_xgb, space=space_xgb, algo=tpe.suggest, max_evals=30, trials=trials)
# print("Best XGBoost params: ", best_params)

## 6. Final Model Training and Prediction

With the best hyperparameters identified, we train our final models on the entire training dataset and generate predictions for the test set.

In [ ]:
# Prepare final datasets with the selected features
X_train = df_sales_train[final_feats].values
X_test = df_sales_test[final_feats].values
y_train = df_sales_train['weekly_sales'].values

# Train and predict with XGBoost
print("Training final XGBoost model...")
xgb_model = xgb.XGBRegressor(**xgb_best_params)
y_pred_xgb = fit_and_predict(xgb_model, X_train, y_train, X_test, offset)
print("XGBoost prediction complete.")

# Train and predict with CatBoost
print("\nTraining final CatBoost model...")
ctb_model = ctb.CatBoostRegressor(**ctb_best_params)
y_pred_ctb = fit_and_predict(ctb_model, X_train, y_train, X_test, offset)
print("CatBoost prediction complete.")

### 6.1. Model Ensembling

To further improve our predictions and reduce variance, we create an ensemble by averaging the predictions from the XGBoost and CatBoost models. This simple approach is often very effective.

In [ ]:
# Average the predictions from both models
y_pred_ensemble = (y_pred_xgb + y_pred_ctb) / 2
print("Ensemble predictions created.")

## 7. Submission

Finally, we format our predictions into the required submission file format.

In [ ]:
# Create submission dataframe
df_sales_test['weekly_sales'] = y_pred_ensemble

# Ensure output directory exists
!mkdir -p ../output

# Save to CSV
submission_path = "../output/mean_xgb_cbt_hyperopt_v1.csv"
df_sales_test[["id", "weekly_sales"]].to_csv(submission_path, index=False)

print(f"Submission file saved to: {submission_path}")
display(df_sales_test[["id", "weekly_sales"]].head())

## 8. Conclusion

This project successfully demonstrates a robust pipeline for sales forecasting. Key takeaways include:
- **The power of feature engineering**: Creating insightful features like rolling statistics and lagged sales from the previous year was crucial for model performance.
- **The effectiveness of gradient boosting**: XGBoost and CatBoost proved to be highly effective for this tabular data problem, and using GPUs significantly sped up training.
- **The benefit of ensembling**: Combining the predictions of multiple diverse models led to a more robust and accurate final prediction.
- **Structured methodology**: A clear, step-by-step process of data prep, feature engineering, importance analysis, tuning, and final modeling was instrumental in achieving a top-3 position in the competition.